In [ ]:
!pip install gym==0.26.2 numpy==1.23.5
import gym, cv2
import tensorflow as tf
from keras.models import load_model
import numpy as np # Import numpy
from google.colab.patches import cv2_imshow # Import cv2_imshow

# Ensure OpenCV is installed
# !pip install opencv-python

env=gym.make('CartPole-v1', render_mode='rgb_array') # Explicitly set the render_mode
q_net = load_model("dqn_q_net.keras")


def policy(state, explore=0.0):
    action = tf.argmax(q_net(state)[0], output_type=tf.int32)
    if tf.random.uniform(shape=(), maxval=1) <= explore:
        action = tf.random.uniform(shape=(), minval=0, maxval=2, dtype=tf.int32) # Changed maxval back to 2 for CartPole actions
    return action


for episode in range(5):
    done = False
    # Ensure env.reset() output is a NumPy array before converting to tensor
    # In newer Gym versions, reset() returns (observation, info)
    reset_output = env.reset()
    # Access the observation from the tuple (first element) and convert to NumPy array
    reset_output_np = np.array(reset_output[0]).reshape(4,) # Access the first element
    state = tf.convert_to_tensor([reset_output_np], dtype=tf.float32) # Specify dtype for consistency

    while not done:
        # Rendering code remains the same
        # Removed the mode argument as it's not supported in newer Gym versions
        frame = env.render()
        # Check if frame is not None before displaying
        if frame is not None:
            # Assuming the default render output is 'rgb_array' and compatible with cv2.imshow
            cv2_imshow(frame) # Replaced cv2.imshow with cv2_imshow
            cv2.waitKey(20)
        else:
            print("Rendering failed. Skipping frame display.")


        action = policy(state)
        # env.step returns a tuple, the state is the first element
        next_state_numpy, reward, done, truncated, info = env.step(action.numpy()) # Added truncated and info for newer Gym versions
        # Convert the next state NumPy array to a tensor with a batch dimension
        state = tf.convert_to_tensor([next_state_numpy], dtype=tf.float32) # Specify dtype

env.close()
# Close all OpenCV windows
cv2.destroyAllWindows()